# Assignment 1: Choosing interpolation method

In this exercise you will check how different interpolation methods
work with different functions, after writing some diagnostic functions.

## Choosing the right interpolation method

Review the code in Lecture 7, part 1 to refresh your memory about

1. Linear interpolation  
1. Quadratic and cubic splines  
1. Polynomial interpolation  


Fix the interpolation window $ x \in [0,25] $, the interpolation nodes
as in the code below, and only change the interpolated function and
interpolation scheme.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import interpolate # Interpolation routines

# fixed parts of the problem
a,b = 0,25 # interval
xd = np.linspace(a,b,1000) # dense grid for plotting
np.random.seed(21234) # fix random number sequences
nodes = np.sort(np.random.uniform(a,b,15)) # sorted random points

## Task 1. Measuring the accuracy

Write a function to plot the true function and the interpolation function,
and let it also return the measure of accuracy equal to the average square deviation
between the two.

More precisely, let two measures of accuracy be calculated: one calculated off
all points within the initial interval, and the second only between the min and max
nodes (so, the second one excludes the extrapolated points).

In [ ]:
# write your code here
# come up with a test of your own

In [ ]:
func=lambda x: np.exp(-x*.1)*np.sin(x*.5) # test function to interpolate

def accuracy(ifunc,nodes=nodes,f=func,xd=xd,color='b'):
    '''helper function to make plots in this lecture'''
    ac1 = ((ifunc(xd)-f(xd))**2).mean()
    xd2 = xd[ np.logical_and(xd>=nodes[0],xd<=nodes[-1]) ]
    ac2 = ((ifunc(xd2)-f(xd2))**2).mean()
    plt.figure(num=1, figsize=(10,8))
    plt.scatter(nodes,f(nodes),color='r') # interpolation data
    plt.plot(xd,f(xd),color='grey') # true function
    plt.plot(xd,ifunc(xd),color=color) # interpolated
    mn = min(np.min(f(xd)),np.min(ifunc(xd2))) # correct ylim
    mm = max(np.max(f(xd2)),np.max(ifunc(xd2)))
    plt.ylim(mn-0.1*np.abs(mn),mm+0.1*np.abs(mm))
    plt.show()
    print('Accuracy measure is',ac1,'without extrapolation it is',ac2)
    return ac1,ac2

accuracy(func) # just testing
accuracy(lambda x: np.full(x.shape,0.4)) # just testing

## Task 2. Interpolating smooth function

Compare the accuracy of all three schemes above to interpolate

$$
f(x) = \exp(-x/10)\sin(x/2)
$$

Hint: use *fill_value=”extrapolate”* option in *interp1d()* to allow for extrapolation

Which interpolation scheme is most accurate?

In [ ]:
# write your code here

In [ ]:
func=lambda x: np.exp(-x*.1)*np.sin(x*.5)

def test_interpolation_methods(func):
    res = {}
    for knd, clr in ('slinear','m'),('quadratic','b'),('cubic','g'),('polynomial','y'):
        if knd == 'polynomial':
            p = np.polynomial.polynomial.polyfit(nodes,func(nodes),len(nodes)-1)
            ifunc = lambda x: np.polynomial.polynomial.polyval(x,p)
        else:
            ifunc = interpolate.interp1d(nodes,func(nodes),kind=knd,fill_value="extrapolate")
        res[knd] = accuracy(ifunc,f=func)
    from pprint import pprint
    print('Accuracy by interpolation method (with and without extrapolations):')
    pprint(res)

test_interpolation_methods(func)

## Task 3. Interpolating complex periodic function

Compare the accuracy of all three schemes above to interpolate

$$
f(x) = x + \exp(x/10)\sin(x)
$$

Which interpolation scheme is most accurate?

In [ ]:
# write your code here

In [ ]:
func=lambda x: x+np.exp(x*.1)*np.sin(x)
test_interpolation_methods(func)

## Task 4. Interpolating function with kinks

Compare the accuracy of all three schemes above to interpolate

$$
f(x) = \max \big( x + \exp(x/10)\sin(3x/4); x + 5  + \exp([x+5]/10)\sin(3[x+5]/4) \big)
$$

Which interpolation scheme is most accurate?

In [ ]:
# write your code here

In [ ]:
func=lambda x: np.maximum( x+np.exp(x*.1)*np.sin(x*.75), x+5+np.exp((x+5)*.1)*np.sin((x+5)*.75) )
test_interpolation_methods(func)

## Task 5. Interpolating discontinuous function

Compare the accuracy of all three schemes above to interpolate

$$
f(x) = \exp(-x/10)\sin(x/2) + \mathbb{1}\{\cos(x)>1/2\}
$$

Which interpolation scheme is most accurate?

In [ ]:
# write your code here

In [ ]:
func=lambda x: np.exp(-x*.1)*np.sin(x*.5)+(np.cos(x)>.5)
test_interpolation_methods(func)